In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial

from IPython.display import HTML

fig, ax = plt.subplots(figsize=[5,5])
line1, = ax.plot([], [], 'ro')

#circle = plt.Circle((0.5, 0.5), 0.1)
#c = ax.add_patch(circle)

#points = ax.plot([],[])

def init():
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    #return points,
    return line1,

def update(frame, ln, x, y):
    x.append(frame)
    y.append(np.sin(frame))
    #ln.set_data(x, y)
    #ln.set_center((frame/2,frame/2))
    ln.set_data([frame/2],[frame/2])
    return ln,

ani = FuncAnimation(
    fig, partial(update, ln=line1, x=[], y=[]),
    frames=np.linspace(-10,10, 12),
    init_func=init, blit=True)

#HTML(ani.to_html5_video())
plt.close(ani._fig)
HTML(ani.to_jshtml())

In [81]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial


# two2danimation object
#
#
class two2danimation(object):
    # __init__
    #
    # initialize the object, with the number of objects
    def __init__(self, nrobjects=1):
        self._time = []
        self._nrobjects = nrobjects
        self._data = []
        for i in range(nrobjects):
            self._data.append({'x': [], 'y': []})
            
    
    
    # append
    #
    # appends the data, the first argument is always the time spot,
    # the next arguments are for each point, x and y floats are necessary!
    def append(self, time, *args):
        self._time.append(time)
        
        if len(args) < (self._nrobjects*2):
            raise ValueError('Too few arguments in append.')
        
        for i in range(self._nrobjects):
            self._data[i]['x'].append(args[(i*2)])
            self._data[i]['y'].append(args[(i*2)+1])                          

    
    # __repr__
    #
    # for printing the data content stored in the object
    def __repr__(self):
        return ','.join([f'time={self._time}', f'data={self._data}'])
    
    
    # _anim_setup
    #
    # internal function to setup the animation frame
    def _anim_setup(self):
        self._fig, self._ax = plt.subplots(figsize=(5,5))
        self._points, = self._ax.plot([], [], 'r.')
        
    
    # _anim_init
    #
    # internal function for the initialisation of the animation
    # content can maybe also set in _anim_setup
    def _anim_init(self):
        self._ax.set_xlim(-10, 10)
        self._ax.set_ylim(-10, 10)
        return self._points,
    
    
    # _anim_update
    #
    # internal function for the animation is called every frame
    # of the animation. The frame is a somewhat time argument,
    # so we need some interpolation of the positional data
    # if the time is not fitting directly to the stored data
    def _anim_update(self, frame, points ):
        xa = []
        ya = []
        # interpolate the position for each object from the
        # stored data
        for i in range(self._nrobjects):
            x = np.interp(frame, self._time, self._data[i]['x'])
            y = np.interp(frame, self._time, self._data[i]['y'])
            xa.append(x)
            ya.append(y)
        points.set_data(xa, ya)
        return points,
    
    
    def animation(self, steps):
        self._anim_setup()
        
        if steps < len(self._time):
            steps = len(self._time)+1
        
        ani = FuncAnimation(
            self._fig, partial(self._anim_update, points=self._points),
            frames=np.linspace(0, self._time[-1], steps),
            #frames=np.arange(0,len(self._time)),
            init_func=self._anim_init, blit=True)
        
        plt.close(ani._fig)
        
        return ani
    

In [82]:
t = two2danimation(1)


time = 0

x = 0
vx = 1
while time < 10:
    x += vx
    if x > 4:
        vx = -vx
    elif x < -4:
        vx = -vx
    #print(x, vx)
    t.append(time, x, 0)
    time += 1

In [83]:
from IPython.display import HTML

ani = t.animation(10)
#plt.close(ani._fig)
HTML(ani.to_jshtml())

In [112]:
from IPython.display import HTML

t = two2danimation(2)


time = 0

x = -10
y = 5
vx = 0.25
vy = 0
m1 = 1
m2 = 100
G = 1
max_step = 0.25
step = max_step

while time < 20:
    x += vx
    y += vy
    
    r = np.sqrt(x**2+y**2)
    a = - G * m2 / (r**2)
    
    # a*t as a vector to the central mass
    # a = np.sqrt(a_x**2+a_y**2)
    # x_r = x / r 
    # y_r = y / r
    # x_r**2 + y_r**2 = 1
    a_x = a * x / r
    a_y = a * y / r
    
    #if np.abs(a) > 1:
    #    step /= 2.
    #    print('step down:', step)
    #else:
    #    if (step*2) < max_step:
    #        step *= 2.
    #        print('step up:', step)
    #    else:
    #        step = max_step

            
        
        
    
    vx += a_x * step
    vy += a_y * step
    
    #print(a, a_x, a_y)
    t.append(time, 0,0, x, y)
    time += step

ani = t.animation(10)
#plt.close(ani._fig)
HTML(ani.to_jshtml())